#### Terms glossary: 
- reusable recipe: data cleaning recipe that includes operations that are more reusable, i.e, the domains of the operations are larger;
- data patterns: in most cases, refer to syntactic level of data values,like regular expression
- split column: this operation is widely used in extracting information on composite data values, by inputting separator that fits all/most of data values 
- golden separator: this separator can fit most of the rows

#### Use Case
1. User U1 works on Dataset D1 (from source S1), use parameter "," to split column `location`;
    - Output a recipe R1 enables D1 fit for use
2. User U2 works on Dataset D2 (from source S2), use parameter ";" to split column `location`;
    - Output a recipe R2 enables D2 fit for use 
3. User U3 works on Integrated D3 (partial data values from source S1, and partial from source S2);
    - The goal is to enable D3 fit for use 

Challenge for User C: 
- Data values from various sources reflecting with different data patterns, with different regular expressions
- Require different repairing methods, i.p. different separators. 

Opportunities for User C:
- Reuse/Inherent reparing functions by integrating R1 and R2
- Detect all potential "Broken" functions and rewrite a more reusable function 

Research Question II: 
- How to make a `perfect` or `reusable` recipe that can deal with larger domains of data values 

### Senerio I:  Same value length, Same order of the semantic data types
- R1 on D1
- R2 on D2

In [8]:
import pandas as pd
df1 = pd.read_csv('Easy/D1.csv')
df1

,Location
0,"Chicago,IL"
1,"Seattle,WA"
2,"Evansville,IN"
3,"Denver,CO"
4,"Hampton,VA"
5,"Richmond,VA"


In [10]:
sep1=','
df_new = df1['Location'].str.split(sep1, expand=True)
df_new

,0,1
0,Chicago,IL
1,Seattle,WA
2,Evansville,IN
3,Denver,CO
4,Hampton,VA
5,Richmond,VA


In [122]:
sep1=','
df1[['city','state']] = df1['Location'].str.split(sep1, expand=True)
df1

,Location,city,state
0,"Chicago,IL",Chicago,IL
1,"Seattle,WA",Seattle,WA
2,"Evansville,IN",Evansville,IN
3,"Denver,CO",Denver,CO
4,"Hampton,VA",Hampton,VA
5,"Richmond,VA",Richmond,VA


In [124]:
df2 = pd.read_csv('Easy/D2.csv')
df2

,Location
0,New York;NY
1,Newark;NJ
2,Overland Park;KS
3,Pomona;CA
4,Portland;OR
5,San Jose;CA


In [125]:
sep2 = ';'
df2[['city','state']] = df2['Location'].str.split(sep2, expand=True)
df2

,Location,city,state
0,New York;NY,New York,NY
1,Newark;NJ,Newark,NJ
2,Overland Park;KS,Overland Park,KS
3,Pomona;CA,Pomona,CA
4,Portland;OR,Portland,OR
5,San Jose;CA,San Jose,CA


### Create D3 by sampling D1 and D2

In [126]:
sample1 = df1['Location'].sample(frac=0.5, replace=False, random_state=1)
sample2 = df2['Location'].sample(frac=0.5, replace=False, random_state=2)

# concatenate the columns by axis=0
concatenated = pd.concat([sample1, sample2], axis=0, keys=['df1', 'df2'])

# convert the concatenated series to a dataframe
comb_df3 = pd.DataFrame(concatenated, columns=['Location'])
comb_df3

Location
df1 2  Evansville,IN
    1     Seattle,WA
    4     Hampton,VA
df2 4    Portland;OR
    1      Newark;NJ
    3      Pomona;CA

Method I: 
- Use parameter sep1 to split column first;
- Do filtering; 
- Use paramter sep2 to split column continuely;


In [134]:
df3_v1 = comb_df3.reset_index(drop=True)
df3_v1

,Location
0,"Evansville,IN"
1,"Seattle,WA"
2,"Hampton,VA"
3,Portland;OR
4,Newark;NJ
5,Pomona;CA


In [135]:
# use sep1=','
df3_v1[['city','state']] = df3['Location'].str.split(sep1, expand=True)
df3_v1

,Location,city,state
0,"Evansville,IN",Evansville,IN
1,"Seattle,WA",Seattle,WA
2,"Hampton,VA",Hampton,VA
3,Portland;OR,Portland;OR,None
4,Newark;NJ,Newark;NJ,None
5,Pomona;CA,Pomona;CA,None


In [136]:
# filter rows that cannot be resolved by sep1
failed_idx = df3_v1[df3_v1['state'].isna()].index.tolist()
failed_idx

[3, 4, 5]

In [137]:
failed_rows = df3_v1.loc[failed_idx].copy()
failed_rows

,Location,city,state
3,Portland;OR,Portland;OR,None
4,Newark;NJ,Newark;NJ,None
5,Pomona;CA,Pomona;CA,None


In [138]:
# use sep2=';'
split_columns = failed_rows['Location'].str.split(sep2, expand=True)

# assign the split columns to the failed_rows DataFrame
failed_rows[['city', 'state']] = split_columns
failed_rows

,Location,city,state
3,Portland;OR,Portland,OR
4,Newark;NJ,Newark,NJ
5,Pomona;CA,Pomona,CA


In [139]:
# replace the incorrect values 
df3_v1.loc[failed_idx] = failed_rows
df3_v1

,Location,city,state
0,"Evansville,IN",Evansville,IN
1,"Seattle,WA",Seattle,WA
2,"Hampton,VA",Hampton,VA
3,Portland;OR,Portland,OR
4,Newark;NJ,Newark,NJ
5,Pomona;CA,Pomona,CA


Method II:
- Scan the whole dataset, profiling: which paramter fit most rows of the dataset?
    - called `golden separator`
- replace all the separators with the golden separator
- split by `golden separator`

In [149]:
df3_v2 = comb_df3.reset_index(drop=True)
df3_v2

,Location
0,"Evansville,IN"
1,"Seattle,WA"
2,"Hampton,VA"
3,Portland;OR
4,Newark;NJ
5,Pomona;CA


In [152]:
# make separator consistent
# replace comma with semicolon
df3_v2['Location'] = df3_v2['Location'].str.replace(sep1, sep2)
df3_v2

,Location
0,Evansville;IN
1,Seattle;WA
2,Hampton;VA
3,Portland;OR
4,Newark;NJ
5,Pomona;CA


In [153]:
df3_v2[['city','state']] = df3_v2['Location'].str.split(sep2, expand=True)
df3_v2

,Location,city,state
0,Evansville;IN,Evansville,IN
1,Seattle;WA,Seattle,WA
2,Hampton;VA,Hampton,VA
3,Portland;OR,Portland,OR
4,Newark;NJ,Newark,NJ
5,Pomona;CA,Pomona,CA


#### Same value length, same order
Takeaway:
- Both Method I and Method II work
- Method II takes fewer steps
- separators can work as `golden separator` either way, no difference.

### Senerio II:  Different value length; Same order of the semantic data types

In [154]:
df4 = pd.read_csv('Medium/D1.csv')

,Location
0,"Chicago,Illinois,US"
1,"Seattle,Washington,US"
2,"Munich,Bavaria,German"
3,"Denver,CO,US"
4,"Guangzhou,Guangdong,China"
5,"Sydney,New South Wales,Australia"


In [155]:
df5 = pd.read_csv('Medium/D2.csv')
df5

,Location
0,Champaign;Illinois
1,Bellevue;Washington
2,Berlin;Brandenburg
3,Chengdu;Sichuan
4,Dubbo;New South Wales


In [157]:
sample1 = df4['Location'].sample(frac=0.5, replace=False, random_state=1)
sample2 = df5['Location'].sample(frac=0.5, replace=False, random_state=2)

# concatenate the columns by axis=0
concatenated = pd.concat([sample1, sample2], axis=0, keys=['df1', 'df2'])

# convert the concatenated series to a dataframe
comb_df6 = pd.DataFrame(concatenated, columns=['Location'])
comb_df6

Location
df1 2      Munich,Bavaria,German
    1      Seattle,Washington,US
    4  Guangzhou,Guangdong,China
df2 2         Berlin;Brandenburg
    4      Dubbo;New South Wales

In [170]:
df6_v1 = comb_df6.copy()
df6_v1 = df6_v1.reset_index(drop=True)
df6_v1

,Location
0,"Munich,Bavaria,German"
1,"Seattle,Washington,US"
2,"Guangzhou,Guangdong,China"
3,Berlin;Brandenburg
4,Dubbo;New South Wales


In [173]:
# Method 1
# use sep1=, to split the column first 
df6_v1[['city','state','country']] = df6_v1['Location'].str.split(sep1, expand=True)
print(df6_v1)
# continue split the failed data values with semicolon 
failed_idx = df6_v1[df6_v1['state'].isna()].index.tolist()
failed_rows = df6_v1.loc[failed_idx].copy()
# use sep2=';'
split_columns = failed_rows['Location'].str.split(sep2, expand=True)
failed_rows[['city', 'state']] = split_columns
df6_v1.loc[failed_idx] = failed_rows
df6_v1

                    Location                   city       state country
0      Munich,Bavaria,German                 Munich     Bavaria  German
1      Seattle,Washington,US                Seattle  Washington      US
2  Guangzhou,Guangdong,China              Guangzhou   Guangdong   China
3         Berlin;Brandenburg     Berlin;Brandenburg        None    None
4      Dubbo;New South Wales  Dubbo;New South Wales        None    None


,Location,city,state,country
0,"Munich,Bavaria,German",Munich,Bavaria,German
1,"Seattle,Washington,US",Seattle,Washington,US
2,"Guangzhou,Guangdong,China",Guangzhou,Guangdong,China
3,Berlin;Brandenburg,Berlin,Brandenburg,None
4,Dubbo;New South Wales,Dubbo,New South Wales,None


In [175]:
# Method II
df6_v2 = comb_df6.copy()
df6_v2 = df6_v2.reset_index(drop=True)
# replace separators to make it consistent 
df6_v2['Location'] = df6_v2['Location'].str.replace(sep1, sep2)
print(df6_v2)
df6_v2[['city','state','country']] = df6_v2['Location'].str.split(sep2, expand=True)
df6_v2

                    Location
0      Munich;Bavaria;German
1      Seattle;Washington;US
2  Guangzhou;Guangdong;China
3         Berlin;Brandenburg
4      Dubbo;New South Wales


,Location,city,state,country
0,Munich;Bavaria;German,Munich,Bavaria,German
1,Seattle;Washington;US,Seattle,Washington,US
2,Guangzhou;Guangdong;China,Guangzhou,Guangdong,China
3,Berlin;Brandenburg,Berlin,Brandenburg,None
4,Dubbo;New South Wales,Dubbo,New South Wales,None


#### Different value length, same order:
Takeaways:
- Use the separator that can split the longest length as `golden separator`

### Senerio III:  Different/Same value length, Different order of the semantic data types

In [177]:
df7 = pd.read_csv('Hard/D1.csv')
df8 = pd.read_csv('Hard/D2.csv')

In [178]:
df7

,Location
0,"US,Chicago,Illinois"
1,"US,Seattle,Washington"
2,"German,Munich,Bavaria"
3,"US,Denver,CO"
4,"China,Guangzhou,Guangdong"
5,"Australia,Sydney,New South Wales"


In [179]:
df8

,Location
0,Illinois;Champaign
1,Washington;Bellevue
2,Brandenburg;Berlin
3,Sichuan;Chengdu
4,New South Wales;Dubbo


In [180]:
sample1 = df7['Location'].sample(frac=0.5, replace=False, random_state=1)
sample2 = df8['Location'].sample(frac=0.5, replace=False, random_state=2)

# concatenate the columns by axis=0
concatenated = pd.concat([sample1, sample2], axis=0, keys=['df1', 'df2'])

# convert the concatenated series to a dataframe
comb_df9 = pd.DataFrame(concatenated, columns=['Location'])
comb_df9

Location
df1 2      German,Munich,Bavaria
    1      US,Seattle,Washington
    4  China,Guangzhou,Guangdong
df2 2         Brandenburg;Berlin
    4      New South Wales;Dubbo

In [189]:
df9 = comb_df9.copy()
df9_v1 = df9.reset_index(drop=True)
df9_v2 = df9.reset_index(drop=True)
df9_v2

,Location
0,"German,Munich,Bavaria"
1,"US,Seattle,Washington"
2,"China,Guangzhou,Guangdong"
3,Brandenburg;Berlin
4,New South Wales;Dubbo


In [190]:
# Method 1: temporary way 
# use sep1=, to split the column first 
df9_v1[['country','city','state']] = df9_v1['Location'].str.split(sep1, expand=True)
print(df9_v1)
# continue split the failed data values with semicolon 
failed_idx = df9_v1[df9_v1['state'].isna()].index.tolist()
failed_rows = df9_v1.loc[failed_idx].copy()
# use sep2=';' order is pre-defined 
split_columns = failed_rows['Location'].str.split(sep2, expand=True)
# misaligned fix 1: ensure the order is correct
failed_rows[['state', 'city']] = split_columns
df9_v1.loc[failed_idx] = failed_rows
print(df9_v1)
# misaligned fix 2: fullfill failed column with blank 
df9_v1.loc[failed_idx, 'country'] = ''
df9_v1

                    Location                country       city       state
0      German,Munich,Bavaria                 German     Munich     Bavaria
1      US,Seattle,Washington                     US    Seattle  Washington
2  China,Guangzhou,Guangdong                  China  Guangzhou   Guangdong
3         Brandenburg;Berlin     Brandenburg;Berlin       None        None
4      New South Wales;Dubbo  New South Wales;Dubbo       None        None
                    Location                country       city  \
0      German,Munich,Bavaria                 German     Munich   
1      US,Seattle,Washington                     US    Seattle   
2  China,Guangzhou,Guangdong                  China  Guangzhou   
3         Brandenburg;Berlin     Brandenburg;Berlin     Berlin   
4      New South Wales;Dubbo  New South Wales;Dubbo      Dubbo   

             state  
0          Bavaria  
1       Washington  
2        Guangdong  
3      Brandenburg  
4  New South Wales  


,Location,country,city,state
0,"German,Munich,Bavaria",German,Munich,Bavaria
1,"US,Seattle,Washington",US,Seattle,Washington
2,"China,Guangzhou,Guangdong",China,Guangzhou,Guangdong
3,Brandenburg;Berlin,,Berlin,Brandenburg
4,New South Wales;Dubbo,,Dubbo,New South Wales


In [196]:
# Advanced Method 1
# use sep1=, to split the column first 
df9_v1_1 = df9.reset_index(drop=True)
df9_v1_1[['country','city','state']] = df9_v1_1['Location'].str.split(sep1, expand=True)
print(df9_v1_1)
# continue split the failed data values with semicolon 
failed_idx = df9_v1_1[df9_v1_1['state'].isna()].index.tolist()
failed_rows = df9_v1_1.loc[failed_idx].copy()
# use sep2=';' order is pre-defined 
split_columns = failed_rows['Location'].str.split(sep2, expand=True)
# misaligned fix 1: ensure the everything is empty for failed columns
df9_v1_1.loc[failed_idx,['country','city','state']] = ''
print(df9_v1_1)
# misaligned fix 2: Ensure the order is correct
failed_rows[['state', 'city']] = split_columns
failed_rows['country'] = ''
df9_v1_1.loc[failed_idx] = failed_rows
df9_v1_1

                    Location                country       city       state
0      German,Munich,Bavaria                 German     Munich     Bavaria
1      US,Seattle,Washington                     US    Seattle  Washington
2  China,Guangzhou,Guangdong                  China  Guangzhou   Guangdong
3         Brandenburg;Berlin     Brandenburg;Berlin       None        None
4      New South Wales;Dubbo  New South Wales;Dubbo       None        None
                    Location country       city       state
0      German,Munich,Bavaria  German     Munich     Bavaria
1      US,Seattle,Washington      US    Seattle  Washington
2  China,Guangzhou,Guangdong   China  Guangzhou   Guangdong
3         Brandenburg;Berlin                               
4      New South Wales;Dubbo                               


,Location,country,city,state
0,"German,Munich,Bavaria",German,Munich,Bavaria
1,"US,Seattle,Washington",US,Seattle,Washington
2,"China,Guangzhou,Guangdong",China,Guangzhou,Guangdong
3,Brandenburg;Berlin,,Berlin,Brandenburg
4,New South Wales;Dubbo,,Dubbo,New South Wales


### Reusing Failure:
Two types of resuability failure:
- Order of column values 
    - Use separator that executes most new columns
- Mismatched values by Re-executing second separator 
``(this is caused by the situation when the missing information [the length of data values are not the same], is not located in the last position)``:
    - Temporary way: replace the failed column values with empty values 
    - Permenant way: reset all column values as empty if the separator does not fit the data values 

In [197]:
# Method II
df9_v2 = comb_df9.copy()
df9_v2 = df9_v2.reset_index(drop=True)
# replace separators to make it consistent 
df9_v2['Location'] = df9_v2['Location'].str.replace(sep1, sep2)
print(df9_v2)
df9_v2[['country','city','state']] = df9_v2['Location'].str.split(sep2, expand=True)
df9_v2
# missing information: country is not located at the last position
# data values are still mismatched 

                    Location
0      German;Munich;Bavaria
1      US;Seattle;Washington
2  China;Guangzhou;Guangdong
3         Brandenburg;Berlin
4      New South Wales;Dubbo


,Location,country,city,state
0,German;Munich;Bavaria,German,Munich,Bavaria
1,US;Seattle;Washington,US,Seattle,Washington
2,China;Guangzhou;Guangdong,China,Guangzhou,Guangdong
3,Brandenburg;Berlin,Brandenburg,Berlin,None
4,New South Wales;Dubbo,New South Wales,Dubbo,None


### Takeaways:
- When both length and order are not the same, give the parameter that can generate longest new columns priority;
- Advanced step:
    - Locate the missing information from the longer data values, replace the corresponding data values with null values at the second resolvation stage;
- Parameter replacement method cannot work here
    - one potential solution is to do patching first, and then replace, finally split 

Quality Evaluator: 